Instala pytorch

In [1]:
pip install torch torchvision


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install rarfile
!apt-get install unrar
import rarfile
import os


rar_path = '/content/drive/MyDrive/IMAGENES_ESQUELETO.rar'
# la carpeta de destino donde se descomprimirá
extract_path = '/content/train'

# Descomprimir el archivo ZIP
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path)

print(f"Descomprimido en: {extract_path}")


rar_path = '/content/drive/MyDrive/IMAGENES_ESQUELETO_prueba.rar'
# la carpeta de destino donde se descomprimirá
extract_path = '/content/test'

# Descomprimir el archivo ZIP
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path)

print(f"Descomprimido en: {extract_path}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Descomprimido en: /content/train
Descomprimido en: /content/test


In [ ]:
#Creando subcarpetas para el entrenamiento
##una clase por letra

In [4]:
import os
import shutil

# Directorio de origen y destino
src_dir = '/content//train/IMAGENES_ESQUELETO'  #  ruta a la carpeta de imágenes originales
dst_dir = '/content/dataset/train'

# Crear la carpeta de destino si no existe
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Mover imágenes a subcarpetas basadas en la letra (clase)
for img_file in os.listdir(src_dir):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Verificar extensiones de imagen
        # Extraer la letra de la clase del nombre del archivo (ej. 'A' de 'A0')
        class_label = img_file[0]  # La primera letra indica la clase

        # Crear la subcarpeta para la letra si no existe
        class_dir = os.path.join(dst_dir, class_label)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        # Mover la imagen a la subcarpeta correspondiente
        shutil.move(os.path.join(src_dir, img_file), os.path.join(class_dir, img_file))

print("Reorganización completada. Las imágenes se han movido a subcarpetas por letra.")

Reorganización completada. Las imágenes se han movido a subcarpetas por letra.


In [5]:
import shutil
##Crear subcarpetas para las pruebas
src_dir = '/content/test/IMAGENES_ESQUELETO_prueba'
dst_dir = '/content/datasetprueba/test'

# Crear la carpeta de destino si no existe
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Mover imágenes a subcarpetas basadas en la letra
for img_file in os.listdir(src_dir):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Verificar extensiones de imagen
        class_label = img_file[0]  # La primera letra indica la clase

        # Crear la subcarpeta para la letra si no existe
        class_dir = os.path.join(dst_dir, class_label)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        # Mover la imagen a la subcarpeta correspondiente
        shutil.move(os.path.join(src_dir, img_file), os.path.join(class_dir, img_file))

print("Reorganización completada para el dataset de prueba.")

Reorganización completada para el dataset de prueba.


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# 1. Hiperparámetros
batch_size = 8
learning_rate = 0.001
num_epochs = 15

# 2. Transformaciones para el dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 3. Cargar el Dataset
train_dataset = datasets.ImageFolder(root='/content/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/datasetprueba/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 4. Definir la ResNet-18
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))  # Ajustar para la detección de lenguaje de señas

# 5. Configuración de entrenamiento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

# 6. Función de entrenamiento
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# 7. Función de validación
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return running_loss / len(loader), accuracy

# 8. Entrenamiento de la red
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, test_loader, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

# 9. Guardar el modelo entrenado
torch.save(model.state_dict(), 'modelo_resnet50_lenguaje_senas.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 195MB/s]


Epoch [1/15], Train Loss: 3.1681, Validation Loss: 3.1357, Accuracy: 13.99%
Epoch [2/15], Train Loss: 2.3532, Validation Loss: 2.9781, Accuracy: 21.37%
Epoch [3/15], Train Loss: 1.7347, Validation Loss: 2.8335, Accuracy: 25.75%
Epoch [4/15], Train Loss: 1.3218, Validation Loss: 2.7036, Accuracy: 30.78%
Epoch [5/15], Train Loss: 0.8998, Validation Loss: 2.5547, Accuracy: 34.31%
Epoch [6/15], Train Loss: 0.6506, Validation Loss: 2.4735, Accuracy: 36.21%
Epoch [7/15], Train Loss: 0.4789, Validation Loss: 2.3640, Accuracy: 38.76%
Epoch [8/15], Train Loss: 0.3600, Validation Loss: 2.3264, Accuracy: 37.58%
Epoch [9/15], Train Loss: 0.2379, Validation Loss: 2.2824, Accuracy: 39.87%
Epoch [10/15], Train Loss: 0.1773, Validation Loss: 2.2433, Accuracy: 39.87%
Epoch [11/15], Train Loss: 0.1729, Validation Loss: 2.1887, Accuracy: 41.44%
Epoch [12/15], Train Loss: 0.1181, Validation Loss: 2.1727, Accuracy: 40.78%
Epoch [13/15], Train Loss: 0.1032, Validation Loss: 2.1782, Accuracy: 41.50%
Epoch [1

In [17]:
import torch

# Cargar el modelo guardado
model_path = 'modelo_resnet50_lenguaje_senas.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  # Cambiar a modo de evaluación

<ipython-input-17-bd83bcb7853e>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from PIL import Image
from torchvision import transforms

# Definir las transformaciones para la imagen de prueba
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar y transformar una imagen de prueba
img_path = '/content/testBadillo/DBad.jpg'
#img_path ='/content/dataset2/test_sorted/D/D1_jpg.rf.89a5f6f41bf8bc795db94105f709dd34.jpg'
image = Image.open(img_path).convert('RGB')
image = transform(image)
image = image.unsqueeze(0)  # Añadir una dimensión para el batch


# Enviar la imagen al dispositivo (GPU o CPU)
image = image.to(device)

# Realizar la predicción
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# Obtener la clase predicha
class_names = train_dataset.classes  # Nombres de las clases
predicted_class = class_names[predicted.item()]

print(f'Predicción: {predicted_class}')

In [11]:
pip install openvino==2023.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 28.5 MB/s eta 0:00:00


In [12]:
import openvino as ov

In [19]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 95.4 MB/s eta 0:00:00


In [25]:
import torch
import torchvision.models as models

# Definir el número de clases
num_classes = 26  # Ajusta según el número de clases en tu dataset

# Cargar el modelo ResNet-50 y ajustar la última capa para el número de clases
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("modelo_resnet50_lenguaje_senas.pth"))
model.eval()

# Crear un tensor de entrada de ejemplo para la exportación (dummy input)
dummy_input = torch.randn(1, 3, 224, 224)  # [batch_size, channels, height, width]

# Exportar a ONNX
torch.onnx.export(
    model,
    dummy_input,
    "modelo_resnet50.onnx",
    opset_version=11,  # Versión de ONNX (11 es generalmente compatible)
    input_names=["input"],  # Nombre de la entrada
    output_names=["output"],  # Nombre de la salida
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}  # Soporte para tamaño de batch variable
)

print("Modelo exportado a ONNX exitosamente como 'modelo_resnet50.onnx'")

<ipython-input-25-1cfb1fb3f88a>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("modelo_resnet18_lenguaje_senas.pth"))


Modelo exportado a ONNX exitosamente como 'modelo_resnet50.onnx'


In [26]:
ov_model = ov.convert_model("modelo_resnet18.onnx",input=[1,3, 224, 224])

In [27]:
ov.save_model(ov_model, 'mnist_3x.xml')